# imports from Python 3

In [1]:
from __future__ import print_function

# CONSTANTS

In [2]:
# Database Access
DB_CONNECT =\
    'jdbc:postgresql://chicagoboothanalytics.clloyfrn47sj.us-west-1.rds.amazonaws.com:5432/ChicagoBoothAnalytics?user=BoothAnalytics&password=123BACmoderator'

# Database Table Names
APP_NAME =\
    'ChicagoBoothAnalytics_app'
DB_TABLE_NAME___USER =\
    'auth_user'
DB_TABLE_NAME___BUSINESSSECTOR =\
    '"%s_businesssector"' % APP_NAME
DB_TABLE_NAME___GEOGREGION =\
    '"%s_geogregion"' % APP_NAME
DB_TABLE_NAME___ORG =\
    '"%s_org"' % APP_NAME
DB_TABLE_NAME___ORG_BUSINESSSECTOR_M2M =\
    '"%s_org_business_sectors"' % APP_NAME
DB_TABLE_NAME___ORG_GEOGREGION_M2M =\
    '"%s_org_geog_regions"' % APP_NAME
DB_TABLE_NAME___USERINTERSTEDINORGS_M2M =\
    '"%s_userinterestedinorgs_orgs"' % APP_NAME
DB_TABLE_NAME___USERINTERSTEDINORGS =\
    '"%s_userinterestedinorgs"' % APP_NAME
DB_TABLE_NAME___PERSON =\
    '"%s_person"' % APP_NAME
DB_TABLE_NAME___ROLELVEL =\
    '"%s_rolelevel"' % APP_NAME
DB_TABLE_NAME___ROLE =\
    '"%s_role"' % APP_NAME
DB_TABLE_NAME___CAREEROPPORTUNITY_GEOGREGION_M2M =\
    '"%s_careeropportunity_geog_regions"' % APP_NAME
DB_TABLE_NAME___CAREEROPPORTUNITY =\
    '"%s_careeropportunity"' % APP_NAME
DB_TABLE_NAME___USERCAREEROPPORTUNITYSTATUS =\
    '"%s_usercareeropportunitystatus"' % APP_NAME
DB_TABLE_NAME___FACTTYPE =\
    '"%s_facttype"' % APP_NAME
DB_TABLE_NAME___ORGFACT =\
    '"%s_orgfact"' % APP_NAME

NB_ROWS_TO_SHOW = 9

# import Spark; initiate SparkContext & HiveContext

In [3]:
if 'pyspark' not in vars():
    from findspark import init
    init()
    import pyspark
    sc = pyspark.SparkContext()
    hc = pyspark.sql.HiveContext(sc)
    
print('SparkContext: %s' % str(sc))
print('HiveContext: %s' % str(hc))

SparkContext: <pyspark.context.SparkContext object at 0x106e51c50>
HiveContext: <pyspark.sql.context.HiveContext object at 0x10836b890>


# ORIGINAL DATABASE TABLES

## "User" table

In [4]:
User_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___USER)\
    .select(
        'id',
        'username',
        'first_name',
        'last_name',
        'email')\
    .sort('username')

User_DDF.registerTempTable('User')

User_DDF.cache()

User_DDF.toPandas()

,id,username,first_name,last_name,email
0,2,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu
1,1,admin,,,BoothAnalyticsClub@gmail.com


## "BusinessSector" table

In [5]:
BusinessSector_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___BUSINESSSECTOR)\
    .sort('name')

BusinessSector_DDF.registerTempTable('BusinessSector')

BusinessSector_DDF.cache()

BusinessSector_DDF.toPandas()

,id,name
0,28,Advertising
1,38,Agriculture
2,37,Audio
3,21,Automotive
4,20,Beverages
5,1,Consulting
6,16,Consumer Goods
7,25,Creative
8,7,E-Commerce
9,29,Education


## "GeogRegion" table

In [6]:
GeogRegion_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___GEOGREGION)\
    .sort('name')

GeogRegion_DDF.registerTempTable('GeogRegion')

GeogRegion_DDF.cache()

GeogRegion_DDF.toPandas()

,id,name
0,8,"Atlanta, GA"
1,13,"Austin, TX"
2,14,"Boston (Greater), MA"
3,6,"Boston, MA"
4,9,"Chicago (Greater), IL"
5,2,"Chicago, IL"
6,11,"Cincinnati, OH"
7,12,"Los Angeles (Greater), LA"
8,10,"Los Angeles, CA"
9,17,"Miami, FL"


## "Org" table

In [7]:
Org_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___ORG)\
    .sort('name')

Org_DDF.registerTempTable('Org')

Org_DDF.cache()

Org_DDF.show(NB_ROWS_TO_SHOW)

+---+---------+
| id|     name|
+---+---------+
| 83|       3M|
| 84|   Abbott|
| 46|Accenture|
| 44|   Acxiom|
|  8| AdYapper|
|  4|   Adatao|
|  7|    Adobe|
|  9|   Affirm|
|132|  AgilOne|
+---+---------+
only showing top 9 rows



## "Org_BusinessSector" table

In [8]:
Org_BusinessSector_m2m_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___ORG_BUSINESSSECTOR_M2M)

Org_BusinessSector_m2m_DDF.registerTempTable('Org_BusinessSector_m2m')

Org_BusinessSector_m2m_DDF.show(NB_ROWS_TO_SHOW)

+---+------+-----------------+
| id|org_id|businesssector_id|
+---+------+-----------------+
|  1|     3|                4|
|  2|     3|                7|
|  6|     5|                4|
|  8|     6|                4|
|  9|     7|                4|
| 10|     8|                4|
| 11|     8|                5|
| 12|     9|                9|
| 13|     9|                4|
+---+------+-----------------+
only showing top 9 rows



In [9]:
Org_BusinessSector_DDF = hc.sql(
    'SELECT \
        Org_BusinessSector_m2m.id as id, \
        Org_BusinessSector_m2m.org_id as org_id, \
        Org_BusinessSector_m2m.businesssector_id as businesssector_id, \
        Org.name as org, \
        BusinessSector.name as business_sector \
    FROM \
        Org_BusinessSector_m2m \
        LEFT JOIN Org ON Org_BusinessSector_m2m.org_id = Org.id \
        LEFT JOIN BusinessSector on Org_BusinessSector_m2m.businesssector_id = BusinessSector.id')\
    .sort('org', 'business_sector')

Org_BusinessSector_DDF.registerTempTable('Org_BusinessSector')

Org_BusinessSector_DDF.cache()

Org_BusinessSector_DDF.show(NB_ROWS_TO_SHOW)

+---+------+-----------------+---------+---------------+
| id|org_id|businesssector_id|      org|business_sector|
+---+------+-----------------+---------+---------------+
|110|    83|               25|       3M|       Creative|
|111|    84|               26|   Abbott|     Healthcare|
|112|    84|               27|   Abbott| Pharmaceutical|
| 68|    46|                1|Accenture|     Consulting|
|113|    44|               28|   Acxiom|    Advertising|
|114|    44|                5|   Acxiom|      Marketing|
| 11|     8|                5| AdYapper|      Marketing|
| 10|     8|                4| AdYapper|     Technology|
|196|     4|                4|   Adatao|     Technology|
+---+------+-----------------+---------+---------------+
only showing top 9 rows



## "Org_GeogRegion" table

In [10]:
Org_GeogRegion_m2m_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___ORG_GEOGREGION_M2M)

Org_GeogRegion_m2m_DDF.registerTempTable('Org_GeogRegion_m2m')

Org_GeogRegion_m2m_DDF.show(NB_ROWS_TO_SHOW)

+---+------+-------------+
| id|org_id|geogregion_id|
+---+------+-------------+
|  1|     3|            1|
|  2|     3|            4|
|  4|     5|            4|
|  5|     6|            3|
|  6|     7|            4|
|  7|     7|            7|
|  8|     9|            3|
|  9|    10|            2|
| 10|    11|            4|
+---+------+-------------+
only showing top 9 rows



In [11]:
Org_GeogRegion_DDF = hc.sql(
    'SELECT \
        Org_GeogRegion_m2m.id as id, \
        Org_GeogRegion_m2m.org_id as org_id, \
        Org_GeogRegion_m2m.geogregion_id as geogregion_id, \
        Org.name as org, \
        GeogRegion.name as geog_region \
    FROM \
        Org_GeogRegion_m2m \
        LEFT JOIN Org ON Org_GeogRegion_m2m.org_id = Org.id \
        LEFT JOIN GeogRegion ON Org_GeogRegion_m2m.geogregion_id = GeogRegion.id')\
    .sort('org', 'geog_region')

Org_GeogRegion_DDF.registerTempTable('Org_GeogRegion')

Org_GeogRegion_DDF.cache()

Org_GeogRegion_DDF.show(NB_ROWS_TO_SHOW)

+---+------+-------------+--------+--------------------+
| id|org_id|geogregion_id|     org|         geog_region|
+---+------+-------------+--------+--------------------+
| 70|    44|            9|  Acxiom|Chicago (Greater)...|
|121|     4|            4|  Adatao|San Francisco Bay...|
|  7|     7|            7|   Adobe|  Salt Lake City, UT|
|  6|     7|            4|   Adobe|San Francisco Bay...|
|  8|     9|            3|  Affirm|   San Francisco, CA|
|116|   132|            4| AgilOne|San Francisco Bay...|
|149|    53|            3|  AirBnB|   San Francisco, CA|
|  9|    10|            2|Allstate|         Chicago, IL|
| 38|    38|            4|Alphabet|San Francisco Bay...|
+---+------+-------------+--------+--------------------+
only showing top 9 rows



## "UserInterestedInOrgs" table

In [12]:
UserInterestedInOrgs_m2m_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___USERINTERSTEDINORGS_M2M)

UserInterestedInOrgs_m2m_DDF.registerTempTable('UserInterestedInOrgs_m2m')

UserInterestedInOrgs_m2m_DDF.show(NB_ROWS_TO_SHOW)

+---+-----------------------+------+
| id|userinterestedinorgs_id|org_id|
+---+-----------------------+------+
|638|                      1|   128|
|639|                      1|   130|
|640|                      1|   131|
|641|                      1|   132|
|642|                      1|     5|
|643|                      1|     7|
|644|                      1|   137|
|645|                      1|   144|
|646|                      1|    20|
+---+-----------------------+------+
only showing top 9 rows



In [13]:
UserInterestedInOrgs_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___USERINTERSTEDINORGS)

UserInterestedInOrgs_DDF.registerTempTable('UserInterestedInOrgs')

UserInterestedInOrgs_DDF = hc.sql(
    'SELECT \
        UserInterestedInOrgs.id as id, \
        UserInterestedInOrgs.user_id as user_id, \
        UserInterestedInOrgs_m2m.org_id as org_id, \
        User.username as username, \
        User.first_name as user_first_name, \
        User.last_name as user_last_name, \
        User.email as user_email_address, \
        Org.name as org_name \
    FROM \
        UserInterestedInOrgs \
        JOIN UserInterestedInOrgs_m2m \
            ON UserInterestedInOrgs.id = UserInterestedInOrgs_m2m.userinterestedinorgs_id \
        JOIN User ON UserInterestedInOrgs.user_id = User.id \
        JOIN Org ON UserInterestedInOrgs_m2m.org_id = Org.id')\
    .sort(
        'username',
        'org_name')

UserInterestedInOrgs_DDF.registerTempTable('UserInterestedInOrgs')

UserInterestedInOrgs_DDF.cache()

UserInterestedInOrgs_DDF.toPandas()

,id,user_id,org_id,username,user_first_name,user_last_name,user_email_address,org_name
0,1,2,4,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Adatao
1,1,2,7,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Adobe
2,1,2,132,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,AgilOne
3,1,2,53,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,AirBnB
4,1,2,124,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Apple
5,1,2,117,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Auction.com
6,1,2,95,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Box
7,1,2,108,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Cloudera
8,1,2,120,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Databricks
9,1,2,144,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Dropbox


## "Person" table

In [14]:
Person_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___PERSON)\
    .selectExpr(
        'id',
        'CONCAT( \
            COALESCE( \
                IF(first_name_alias = "", NULL, first_name_alias), \
                first_name), \
            " ", \
            UPPER(last_name)) \
            as name',
        'last_name',
        'first_name',
        'first_name_alias',
        'IF(gender, "male", "female") as gender')\
    .sort('last_name')

Person_DDF.registerTempTable('Person')

Person_DDF.cache()

Person_DDF.show(NB_ROWS_TO_SHOW)

+---+--------------+---------+--------------+----------------+------+
| id|          name|last_name|    first_name|first_name_alias|gender|
+---+--------------+---------+--------------+----------------+------+
| 21|  Paul ARONSON|  Aronson|          Paul|                |  male|
|  2|   Sahil BAHRI|    Bahri|         Sahil|                |  male|
| 18|  Anju BHALDAR|  Bhaldar|          Anju|                |female|
| 16|Rudi BONAPARTE|Bonaparte|          Rudi|                |  male|
| 25|     Greg BUSH|     Bush|       Gregory|            Greg|  male|
|  8|    Cheng CHEN|     Chen|         Cheng|                |  male|
| 11| John CIASULLI| Ciasulli|    John David|            John|  male|
|  7|Amanda COLLINS|  Collins|Amanda Marlene|          Amanda|female|
| 12|       Tony DO|       Do|         Thong|            Tony|  male|
+---+--------------+---------+--------------+----------------+------+
only showing top 9 rows



# "RoleLevel" table

In [15]:
RoleLevel_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___ROLELVEL)\
    .sort('level_number_from_high_to_low')

RoleLevel_DDF.registerTempTable('RoleLevel')
    
RoleLevel_DDF.show(NB_ROWS_TO_SHOW)

+---+------+-----------------------------+
| id| level|level_number_from_high_to_low|
+---+------+-----------------------------+
|  2| Board|                            1|
|  1|     C|                            2|
|  3|senior|                            3|
|  4|   mid|                            4|
|  5|junior|                            5|
+---+------+-----------------------------+



## "Role" table

In [16]:
Role_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___ROLE)

Role_DDF.registerTempTable('Role')

Role_DDF = hc.sql(
    'SELECT \
        Role.id as id, \
        CONCAT(Role.title, " [", RoleLevel.level, "-level]") as title, \
        RoleLevel.level_number_from_high_to_low as level \
    FROM \
        Role \
        JOIN RoleLevel ON Role.level_id = RoleLevel.id')\
    .sort('level_number_from_high_to_low', 'title')

Role_DDF.registerTempTable('Role')

Role_DDF.cache()
    
Role_DDF.show(NB_ROWS_TO_SHOW)

+---+--------------------+-----+
| id|               title|level|
+---+--------------------+-----+
|  2|Chair of the Boar...|    1|
|  5|Director of the B...|    1|
|  8|(Co-)Founder [C-l...|    2|
| 19|Chief Analytics O...|    2|
|  1|Chief Executive O...|    2|
| 20|Chief Financial O...|    2|
| 21|Chief Operating O...|    2|
| 18|Chief Technology ...|    2|
| 86|Chief Architect, ...|    3|
+---+--------------------+-----+
only showing top 9 rows



## "CareerOpportunity_GeogRegion" table

In [17]:
CareerOpportunity_GeogRegion_m2m_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___CAREEROPPORTUNITY_GEOGREGION_M2M)

CareerOpportunity_GeogRegion_m2m_DDF.registerTempTable('CareerOpportunity_GeogRegion_m2m')

CareerOpportunity_GeogRegion_m2m_DDF.show(NB_ROWS_TO_SHOW)

+---+--------------------+-------------+
| id|careeropportunity_id|geogregion_id|
+---+--------------------+-------------+
|  1|                   5|            2|
|  2|                   3|            3|
|  3|                   1|            3|
|  4|                   2|            4|
|  5|                   4|            3|
|  6|                   6|            1|
|  7|                   6|            4|
|  9|                   7|            4|
| 12|                   9|            4|
+---+--------------------+-------------+
only showing top 9 rows



In [18]:
CareerOpportunity_GeogRegion_DDF = hc.sql(
    'SELECT \
        CareerOpportunity_GeogRegion_m2m.careeropportunity_id as careeropportunity_id, \
        GeogRegion.name as geog_region \
    FROM \
        CareerOpportunity_GeogRegion_m2m \
        LEFT JOIN GeogRegion ON CareerOpportunity_GeogRegion_m2m.geogregion_id = GeogRegion.id')

CareerOpportunity_GeogRegion_DDF.registerTempTable('CareerOpportunity_GeogRegion')

CareerOpportunity_GeogRegion_DDF.cache()

CareerOpportunity_GeogRegion_DDF.show(NB_ROWS_TO_SHOW)

+--------------------+--------------------+
|careeropportunity_id|         geog_region|
+--------------------+--------------------+
|                   5|         Chicago, IL|
|                   3|   San Francisco, CA|
|                   1|   San Francisco, CA|
|                   2|San Francisco Bay...|
|                   4|   San Francisco, CA|
|                   6|         Seattle, WA|
|                   6|San Francisco Bay...|
|                   7|San Francisco Bay...|
|                   9|San Francisco Bay...|
+--------------------+--------------------+
only showing top 9 rows



In [20]:
CareerOpportunity_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___CAREEROPPORTUNITY)

CareerOpportunity_DDF.registerTempTable('CareerOpportunity')

CareerOpportunity_DDF = hc.sql(
    'SELECT \
        CareerOpportunity.id as id, \
        Org.id as org_id, \
        Org.name as org, \
        Role.title as role, \
        Role.level as role_level, \
        CareerOpportunity_GeogRegion.geog_region as geog_region, \
        CareerOpportunity.open as open, \
        CareerOpportunity.posting_date as posting_date \
    FROM \
        CareerOpportunity \
        LEFT JOIN Org ON CareerOpportunity.org_id = Org.id \
        LEFT JOIN Role ON CareerOpportunity.role_id = Role.id \
        LEFT JOIN CareerOpportunity_GeogRegion ON \
            CareerOpportunity.id = CareerOpportunity_GeogRegion.careeropportunity_id')\
    .sort(
        'org',
        'role_level',
        'role',
        'geog_region')\
    .sort('open', ascending=False)

CareerOpportunity_DDF.registerTempTable('CareerOpportunity')

CareerOpportunity_DDF.cache()

CareerOpportunity_DDF.show(NB_ROWS_TO_SHOW)

+---+------+--------------------+--------------------+----------+--------------------+----+------------+
| id|org_id|                 org|                role|role_level|         geog_region|open|posting_date|
+---+------+--------------------+--------------------+----------+--------------------+----+------------+
|  8|     4|              Adatao|Data Scientist [m...|         4|San Francisco Bay...|true|  2015-10-12|
| 21|     7|               Adobe|Data Scientist [m...|         4|San Francisco Bay...|true|  2015-10-06|
| 58|   132|             AgilOne|Senior Data Scien...|         4|San Francisco Bay...|true|  2015-11-01|
| 88|    53|              AirBnB|Data Scientist - ...|         4|   San Francisco, CA|true|  2015-11-10|
| 91|    53|              AirBnB|Data Scientist Ex...|         4|   San Francisco, CA|true|  2015-10-25|
| 89|    53|              AirBnB|Global Mobilizati...|         4|   San Francisco, CA|true|  2015-10-15|
| 90|    53|              AirBnB|Product Analyst [...| 

## "UserCareerOpportunityStatus" table

In [21]:
UserCareerOpportunityStatus_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___USERCAREEROPPORTUNITYSTATUS)
    
UserCareerOpportunityStatus_DDF.registerTempTable('UserCareerOpportunityStatus')

UserCareerOpportunityStatus_DDF = hc.sql(
    'SELECT \
        UserCareerOpportunityStatus.id as id, \
        UserCareerOpportunityStatus.user_id as user_id, \
        UserCareerOpportunityStatus.career_opportunity_id as careeropportunity_id, \
        User.username as username, \
        User.first_name as user_first_name, \
        User.last_name as user_last_name, \
        User.email as user_email, \
        UserCareerOpportunityStatus.status as status \
    FROM \
        UserCareerOpportunityStatus \
        JOIN User ON UserCareerOpportunityStatus.user_id = User.id')

UserCareerOpportunityStatus_DDF.registerTempTable('UserCareerOpportunityStatus')

UserCareerOpportunityStatus_DDF.cache()

UserCareerOpportunityStatus_DDF.toPandas()

,id,user_id,careeropportunity_id,username,user_first_name,user_last_name,user_email,status
0,1,2,21,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,applied; LinkedIn messaged to recruiter; TODO:...
1,2,2,20,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,applied
2,3,2,26,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,applied
3,4,2,29,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,applied; to talk to Luan's friend
4,5,2,31,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,applied; to find recruiter
5,6,2,32,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Applied
6,7,2,33,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Initiated convers; no visa sponsorship
7,8,2,17,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Applied; heard nothing
8,9,2,56,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Applied;
9,10,2,57,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Applied;


## "FactType" table

In [22]:
FactType_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___FACTTYPE)\
    .sort('label')
    
FactType_DDF.registerTempTable('FactType')

FactType_DDF.toPandas()

,id,label
0,3,LinkedIn profile URL
1,6,business school
2,5,business/work phone number
3,8,careers website
4,1,email address
5,2,hobby
6,4,website
7,7,year founded


## "OrgFact" table

In [24]:
OrgFact_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___ORGFACT)

OrgFact_DDF.registerTempTable('OrgFact')

OrgFact_DDF = hc.sql(
    'SELECT \
        OrgFact.id as id, \
        OrgFact.org_id as org_id, \
        OrgFact.fact_type_id as facttype_id, \
        Org.name as org, \
        FactType.label as fact_type, \
        OrgFact.fact as fact \
    FROM \
        OrgFact \
        LEFT JOIN Org ON OrgFact.org_id = Org.id \
        LEFT JOIN FactType ON OrgFact.fact_type_id = FactType.id')\
    .sort(
        'org',
        'fact_type',
        'fact')

OrgFact_DDF.registerTempTable('OrgFact')

OrgFact_DDF.cache()

OrgFact_DDF.show(NB_ROWS_TO_SHOW)

+---+------+-----------+---------+---------------+--------------------+
| id|org_id|facttype_id|      org|      fact_type|                fact|
+---+------+-----------+---------+---------------+--------------------+
| 85|    83|          4|       3M|        website|              3M.com|
| 86|    84|          4|   Abbott|        website|          Abbott.com|
| 47|    46|          4|Accenture|        website|       Accenture.com|
| 45|    44|          4|   Acxiom|        website|          Acxiom.com|
| 14|     8|          4| AdYapper|        website|        AdYapper.com|
|159|     4|          8|   Adatao|careers website|www.linkedin.com/...|
|  1|     4|          4|   Adatao|        website|          Adatao.com|
|  4|     4|          7|   Adatao|   year founded|                2012|
| 13|     7|          4|    Adobe|        website|           Adobe.com|
+---+------+-----------+---------+---------------+--------------------+
only showing top 9 rows



# DERIVED TABLES

## "OrgOverview" table

In [27]:
# OrgFact_website table
FactType_Website_DDF = hc.sql(
    'SELECT \
        id, \
        label \
    FROM \
        FactType \
    WHERE \
        label = "website"')

OrgFact_Website_DDF =\
    OrgFact_DDF.join(
        FactType_Website_DDF,
        on=OrgFact_DDF.facttype_id == FactType_Website_DDF.id)\
    .select(
        'org_id',
        'fact')
    
OrgFact_Website_DDF.registerTempTable('OrgFact_Website')


# OrgFact_CareersWebsite table

FactType_CareersWebsite_DDF = hc.sql(
    'SELECT \
        id, \
        label \
    FROM \
        FactType \
    WHERE \
        label = "careers website"')

OrgFact_CareersWebsite_DDF =\
    OrgFact_DDF.join(
        FactType_CareersWebsite_DDF,
        on=OrgFact_DDF.facttype_id == FactType_CareersWebsite_DDF.id)\
    .select(
        'org_id',
        'fact')
    
OrgFact_CareersWebsite_DDF.registerTempTable('OrgFact_CareersWebsite')


# OrgNbOpenCareerOpportunities
OrgNbOpenCareerOpportunities_DDF = hc.sql(
    'SELECT \
        org_id, \
        COUNT(*) as nb_open_career_opportunities \
    FROM \
        CareerOpportunity \
    WHERE \
        open = true \
    GROUP BY \
       org_id')

OrgNbOpenCareerOpportunities_DDF.registerTempTable('OrgNbOpenCareerOpportunities')


# OrgOverview table
OrgOverview_DDF = hc.sql(
    'SELECT \
        Org.id as id, \
        Org.name as name, \
        COLLECT_SET(OrgFact_Website.fact) as websites, \
        COLLECT_SET(OrgFact_CareersWebsite.fact) as careers_websites, \
        COLLECT_SET(Org_BusinessSector.business_sector) as business_sectors, \
        COLLECT_SET(Org_GeogRegion.geog_region) as geog_regions, \
        OrgNbOpenCareerOpportunities.nb_open_career_opportunities as nb_open_career_opportunities \
    FROM \
        Org \
        LEFT JOIN OrgFact_Website ON Org.id = OrgFact_Website.org_id \
        LEFT JOIN OrgFact_CareersWebsite ON Org.id = OrgFact_CareersWebsite.org_id \
        LEFT JOIN Org_BusinessSector ON Org.id = Org_BusinessSector.org_id \
        LEFT JOIN Org_GeogRegion ON Org.id = Org_GeogRegion.org_id \
        LEFT JOIN OrgNbOpenCareerOpportunities ON Org.id = OrgNbOpenCareerOpportunities.org_id \
    GROUP BY \
        Org.id, \
        Org.name, \
        OrgNbOpenCareerOpportunities.nb_open_career_opportunities')\
    .sort('name')

OrgOverview_DDF.toPandas()

,id,name,websites,careers_websites,business_sectors,geog_regions,nb_open_career_opportunities
0,83,3M,[3M.com],[],[Creative],[],NaN
1,84,Abbott,[Abbott.com],[],"[Healthcare, Pharmaceutical]",[],NaN
2,46,Accenture,[Accenture.com],[],[Consulting],[],NaN
3,44,Acxiom,[Acxiom.com],[],"[Advertising, Marketing]","[Chicago (Greater), IL]",NaN
4,8,AdYapper,[AdYapper.com],[],"[Marketing, Technology]",[],NaN
5,4,Adatao,[Adatao.com],[www.linkedin.com/vsearch/j?f_C=3031699],[Technology],"[San Francisco Bay Area, CA]",1
6,7,Adobe,[Adobe.com],[],[Technology],"[Salt Lake City, UT, San Francisco Bay Area, CA]",1
7,9,Affirm,[Affirm.com],[],"[Financial Services: Consumer Finance, Technol...","[San Francisco, CA]",NaN
8,132,AgilOne,[AgilOne.com],[],"[Marketing, Technology, Technology: Data & Ana...","[San Francisco Bay Area, CA]",1
9,53,AirBnB,[AirBnB.com],[www.linkedin.com/vsearch/j?f_C=309694],"[Hospitality, Technology]","[San Francisco, CA]",4


## "MyCareerOpportunity" tables

In [28]:
InterestingOrg_CareersWebsites_DDF = hc.sql(
    'SELECT \
        UserInterestedInOrgs.org_name as name, \
        OrgFact_CareersWebsite.fact as careers_website \
    FROM \
        UserInterestedInOrgs \
        LEFT JOIN OrgFact_CareersWebsite \
            ON UserInterestedInOrgs.org_id = OrgFact_CareersWebsite.org_id \
    WHERE \
        UserInterestedInOrgs.username = "MBALearnsToCode"')\
    .sort('name')

InterestingOrg_CareersWebsites_DDF.toPandas()

,name,careers_website
0,Adatao,www.linkedin.com/vsearch/j?f_C=3031699
1,Adobe,None
2,AgilOne,None
3,AirBnB,www.linkedin.com/vsearch/j?f_C=309694
4,Apple,None
5,Auction.com,www.linkedin.com/vsearch/j?f_C=1474032
6,Box,Box.com/careers
7,Cloudera,None
8,Databricks,Databricks.com/company/careers
9,Databricks,www.Glassdoor.com/Job/Databricks-Jobs-E954734.htm


In [30]:
MyCareerOpportunity_DDF = hc.sql(
    'SELECT \
        CareerOpportunity.org as org, \
        CareerOpportunity.role as role, \
        COLLECT_SET(CareerOpportunity.geog_region) as geog_regions, \
        CareerOpportunity.open as open, \
        UserCareerOpportunityStatus.status as status \
    FROM \
        UserInterestedInOrgs \
        JOIN CareerOpportunity ON UserInterestedInOrgs.org_id = CareerOpportunity.org_id \
        LEFT JOIN UserCareerOpportunityStatus ON \
            CareerOpportunity.id = UserCareerOpportunityStatus.careeropportunity_id \
    WHERE \
        UserInterestedInOrgs.username = "MBALearnsToCode" OR \
        UserCareerOpportunityStatus.username ="MBALearnsToCode" \
    GROUP BY \
        CareerOpportunity.id, \
        CareerOpportunity.org, \
        CareerOpportunity.role, \
        CareerOpportunity.open, \
        UserCareerOpportunityStatus.status \
    ORDER BY \
        CareerOpportunity.org, \
        CareerOpportunity.role, \
        NOT CareerOpportunity.open')

MyCareerOpportunity_DDF.toPandas()

,org,role,geog_regions,open,status
0,Adatao,Data Scientist [mid-level],"[San Francisco Bay Area, CA]",True,Ongoing
1,Adobe,Data Scientist [mid-level],"[San Francisco Bay Area, CA]",True,applied; LinkedIn messaged to recruiter; TODO:...
2,AgilOne,Senior Data Scientist - Client Success [mid-le...,"[San Francisco Bay Area, CA]",True,None
3,AirBnB,Data Scientist - Economic Policy [mid-level],"[San Francisco, CA]",True,None
4,AirBnB,"Data Scientist Expert, Brand & Advertising [mi...","[San Francisco, CA]",True,None
5,AirBnB,Global Mobilization Data Researcher [mid-level],"[San Francisco, CA]",True,None
6,AirBnB,Product Analyst [mid-level],"[San Francisco, CA]",True,None
7,Apple,Data Scientist - Applied Machine Learning [mid...,"[San Francisco Bay Area, CA]",True,None
8,Apple,Data Scientist [mid-level],"[San Francisco Bay Area, CA]",True,None
9,Apple,"Machine Learning Data Scientist, Siri [mid-level]","[San Francisco Bay Area, CA]",True,None


# _end._